In [1]:
import tensorflow as tf
from tensorflow.contrib.slim.nets import inception
import matplotlib.pyplot as plt
import inception_preprocessing
from datasets import imagenet

In [6]:
# image_size = 299
image_size = inception.inception_v1.default_image_size
test_img = tf.image.decode_jpeg("data_seq/Basketball/img/0001.jpg", channels=3)
processed_image = inception_preprocessing.preprocess_image(test_img, image_size, image_size, is_training=False)
processed_images = tf.expand_dims(processed_image, 0)

In [7]:
slim = tf.contrib.slim
with tf.Graph().as_default():

    with slim.arg_scope(inception.inception_v1_arg_scope()):
        logits, _ = inception.inception_v1(processed_images, num_classes=1001, is_training=False)
    probabilities = tf.nn.softmax(logits)
    init_fn = slim.assign_from_checkpoint_fn("inception_v1.ckpt",
                                             slim.get_model_variables("InceptionV1"))
    with tf.Session() as sess:
        init_fn(sess)
        np_image, probabilities = sess.run([test_img, probabilities])

probabilities = probabilities[0, 0:]
sorted_inds = [i[0] for i in sorted(enumerate(-probabilities), key=lambda x: x[1])]

names = imagenet.create_readable_names_for_imagenet_labels()
for i in range(5):
    index = sorted_inds[i]
    print('Probability %0.2f%% => [%s]' % (probabilities[index], names[index]))

ValueError: Variable InceptionV1/Conv2d_1a_7x7/weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/usr/local/lib/python3.4/dist-packages/tensorflow/contrib/framework/python/ops/variables.py", line 208, in variable
    caching_device=caching_device)
  File "/usr/local/lib/python3.4/dist-packages/tensorflow/contrib/framework/python/ops/arg_scope.py", line 177, in func_with_args
    return func(*args, **current_args)
  File "/usr/local/lib/python3.4/dist-packages/tensorflow/contrib/framework/python/ops/variables.py", line 244, in model_variable
    caching_device=caching_device, device=device)
